In [1]:
import pandas as pd
from nltk import word_tokenize
import nltk
from nltk.util import ngrams
from collections import Counter
import numpy as np
from fuzzywuzzy import fuzz
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 100000)
pd.set_option('display.max_rows', 500)
import string

In [2]:
%reload_kedro

2019-11-19 20:21:57,693 - root - INFO - ** Kedro project sneakairs
2019-11-19 20:21:57,694 - root - INFO - Defined global variable context


In [3]:
df = context.io.load('kof_features')

2019-11-19 20:21:57,709 - kedro.io.data_catalog - INFO - Loading data from `kof_features` (CSVLocalDataSet)...


In [4]:
df.head(5)

,Unnamed: 0,Unnamed: 0.1,Code_Style,Name,Brand,Date,Retail_Price,Colorway,Story,KOF_Wants,Avg_Resale,Net_Profit,Purchase,Brand_Code,Color_Code,Womens,bCollab,OG,SP,QS,SB,LS,NRG,PRM,NSW,RETRO,SE,PE,GS,HS
0,0,0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air Foamposite One comes covered in a black and sail snakeskin textured material with black leather detailing on the eyestay that wraps all around the ankle. Additional black detailing can be seen on the tongue, laces, and inner liner while Habanero Red lands on the varios branding throughout the shoe, running down the tongue, on the pull tabs, and lastly on the rubber outsole.",2281,305,49.545275,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,BV4580-400,Undercover x Nike SFB Mountain Obsidian University Red,Nike,2019-04-15,350,Obsidian/University Red-Dark Obsidian,"The Undercover x Nike SFB Mountain Obsidian/University Red is a collaboration from designer Jun Takahashi’s and Nike. Adding his own spin with an additional toggle lacing system and nylon straps, the silhouette is also detailed with “The New Warriors” branded across the straps. Finally, Undercover logos on the heels, co-branding on the insoles and its usual mesh upper and rugged outsole completes the design.",37,192,-186.495960,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,2,BV4580-001,Undercover x Nike SFB Mountain Black,Nike,2019-04-15,350,Black/Sail-Black,"The Undercover x Nike SFB Mountain Black is Introduced in a mixture of Black/Sail-Black. Undercover designer Jun Takahashi adds his twist to the design with an additional toggle lacing system and nylon straps. Another detail includes “The New Warriors” branded across the straps with Undercover logos on the heels, followed by co-branding on the insoles. Finally, its mesh upper and rugged outsole tops it all off.",70,231,-150.177405,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,3,AO2607-900,Nike PG 3 Mamba Mentality,Nike,2019-04-13,110,Multi-Color/Opti Yellow,"The Nike PG 3 Mamba Mentality comes covered in a black, yellow, grey, tan upper with a geometric pattern dominating a large portion of the shoe. Black overlays on the midfoot and heel keep the momentum going with large gold Swooshes landing on the lateral midsole. Paul George branding on the tongue and heels along with a yellow rubber outsole complete the look.",179,109,-16.829295,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,4,AO2918-102,Nike Kyrie 5 Mamba Mentality,Nike,2019-04-13,130,White/Cyber,"The Nike Kyrie 5 Mamba Mentality is a new rendition of Kyrie Irving’s signature silhouette that will be celebrating Mamba Day 2019. Covered in a color scheme of White and Cyber, the shoe sports neutral uppers with a hint of snakeskin detailing as a nod to Kobe Bryant’s alternate ego. Finally, hits of purple and neon green, along with red speckling on the heel and midsole are applied as a tribute to the 2009 Joker-inspired “Chaos” Zoom Kobe 5.\r\n\r\nGS pictured above.",271,107,-39.271785,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
len(df)

6338

## TF-IDF on sneaker names

In [6]:
df['name'] = df.Name.str.lower()

In [8]:
names = list(df.name)

In [9]:
len(names)

6338

In [10]:
def tuple2string(tup): 
    str =  ' '.join(tup) 
    return str

In [11]:
grams = {}
for i in range(len(names)):
    all_grams = []
    for j in range(3,7):
        shoe = names[i]
        shoe = shoe.replace('nike ','')
        shoe = shoe.replace(' x ', '')
        shoe = shoe.replace('adidas', '')
        shoe = shoe.replace('og', '')
        shoe = shoe.replace('wmns', '')
        shoe = shoe.replace('high', '')
        shoe = shoe.replace('low', '')
        shoe = shoe.replace('gs', '')
        shoe = shoe.replace('mid', '')
        shoe = shoe.replace('retro', '')
        shoe = shoe.replace('footwear', '')
        shoe = shoe.replace('black', '')
        shoe = shoe.replace('white', '')
        shoe = shoe.translate(str.maketrans('','',string.punctuation))
        
        
        ngrams_raw = list(ngrams(shoe.split(), j))
        ngrams_clean = [tuple2string(ngrams_raw[i]) for i in range(len(ngrams_raw))]
        all_grams.extend(ngrams_clean)
    grams[names[i]] = all_grams

In [12]:
score_grams = []
for keys in grams:
    for j in range(len(grams[keys])):
        score = fuzz.ratio(keys, grams[keys][j])
        score_grams.append([keys, grams[keys][j], score])

In [13]:
silhoutte = pd.DataFrame(score_grams, columns = ['name', 'ngram', 'score'])

In [14]:
silhoutte

,name,ngram,score
0,nike air foamposite one snakeskin,air foamposite one,71
1,nike air foamposite one snakeskin,foamposite one snakeskin,84
2,nike air foamposite one snakeskin,air foamposite one snakeskin,92
3,undercover x nike sfb mountain obsidian university red,undercoversfb mountain obsidian,73
4,undercover x nike sfb mountain obsidian university red,mountain obsidian university,68
...,...,...,...
34172,air jordan 11 concord og (1995),jordan 11 concord,71
34173,air jordan 11 concord og (1995),11 concord 1995,65
34174,air jordan 11 concord og (1995),air jordan 11 concord,81
34175,air jordan 11 concord og (1995),jordan 11 concord 1995,83


In [15]:
avg_ngram_scores = silhoutte.groupby(['ngram'])['score'].mean().reset_index(name='avg_score')
num_ngrams = silhoutte.groupby(['ngram'])['score'].count().reset_index(name='total')

In [16]:
silhoutte = pd.merge(silhoutte, num_ngrams, how='left', on='ngram')

In [17]:
silhoutte = pd.merge(silhoutte, avg_ngram_scores, how='left', on='ngram')

In [18]:
silhoutte = silhoutte[silhoutte['total'] > 1]
silhoutte = silhoutte.drop(columns=['score'])

In [19]:
silhoutte = silhoutte[silhoutte.avg_score<silhoutte.groupby('name').avg_score.transform('quantile',0.95)]
silhoutte = silhoutte[silhoutte.avg_score>silhoutte.groupby('name').avg_score.transform('quantile',0.01)]

In [20]:
# silhoutte = silhoutte[silhoutte.total<silhoutte.groupby('name').total.transform('quantile',0.99)]
# silhoutte = silhoutte[silhoutte.total>silhoutte.groupby('name').total.transform('quantile',0.01)]

In [21]:
silhoutte = silhoutte.sort_values(['total', 'avg_score'], ascending=False).drop_duplicates(['name'])


In [22]:
silhoutte.head(500)

,name,ngram,total,avg_score
2398,nike wmns air force 1 rebel xx gym red,air force 1,247,43.218623
2413,nike wmns air force 1 rebel xx praline,air force 1,247,43.218623
2423,nike wmns air force 1 rebel xx blue void,air force 1,247,43.218623
2665,nike wmns air force 1 xx star studded black,air force 1,247,43.218623
2675,nike wmns air force 1 xx star studded white,air force 1,247,43.218623
2920,nike wmns air force 1 07 smokey mauve,air force 1,247,43.218623
3037,nike air force 1 07 lv8 utility volt,air force 1,247,43.218623
4225,nike wmns air force 1 jester high xx black dusty peach,air force 1,247,43.218623
4239,nike wmns air force 1 jester high xx violet ash,air force 1,247,43.218623
4793,nike air force 1 low gs silt red,air force 1,247,43.218623


In [32]:
silhoutte = silhoutte.rename(columns={'ngram':'silhoutte'})
merge_silhoutte = silhoutte[['name','silhoutte', 'avg_score']]

In [33]:
df_silhouttes = pd.merge(df, merge_silhoutte, how='left', on='name')

In [39]:
df_silhouttes.to_csv('/Users/nikhilbhargava/Documents/sneakairs/data/07_model_output/silhoutte-active-learning.csv', index=False)